# Your Title Here

**Name(s)**: (your name(s) here)

**Website Link**: (your website link)

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from scipy import stats
import plotly.express as px
pd.options.plotting.backend = 'plotly'
dataloc = Path('data')
data_raw = pd.read_excel(dataloc / 'outage.xlsx.xls')

# from dsc80_utils import * # Feel free to uncomment and use this.

## Step 1: Introduction

I'm perticularly interested in the the number of weather related outaged over time, and if the effects of global warming can be seen in this dataset using the weather related outages as a proxy. I'm also interested in if there is a correlation between the population density of an area and things like outage duration and frequency. this is much harder to study since the data is grouped by state and not the locaiton where the outage occured.

## Step 2: Data Cleaning and Exploratory Data Analysis

Format all of the data and modify time columns to encode all data in correct format for analysis.

In [2]:
formated_data = pd.DataFrame(data_raw.iloc[6:, 1:].to_numpy(), columns=data_raw.iloc[4, 1:])
temp_data = formated_data.dropna(subset=['OBS', 'OUTAGE.START.DATE', 'OUTAGE.START.TIME', 'OUTAGE.RESTORATION.DATE', 'OUTAGE.RESTORATION.TIME'])

cols_to_str = ['OUTAGE.START.DATE', 'OUTAGE.START.TIME', 'OUTAGE.RESTORATION.DATE', 'OUTAGE.RESTORATION.TIME']
for col in cols_to_str:
    temp_data.loc[:, col] = temp_data.loc[:, col].astype(str)

temp_data.loc[:, 'outageStart'] = pd.to_datetime(temp_data['OUTAGE.START.DATE'] + ' ' + temp_data['OUTAGE.START.TIME'])
temp_data.loc[:, 'outageEnd'] = pd.to_datetime(temp_data['OUTAGE.RESTORATION.DATE'] + ' ' + temp_data['OUTAGE.RESTORATION.TIME'])

temp_data = temp_data[['OBS', 'outageStart', 'outageEnd']]
formated_data = formated_data.merge(temp_data, left_on='OBS', right_on='OBS', how='left')

formated_data = formated_data.infer_objects()

/Users/martinhawks/miniconda3/envs/dsc80/lib/python3.8/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
/Users/martinhawks/miniconda3/envs/dsc80/lib/python3.8/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


Distribution of the proportion of gdp each state is responsible for in the US. This value might have some correlation with number of outages or outage response time, so I want to understand how it works before I start my analysis.

In [3]:
px.bar(formated_data.groupby('POSTAL.CODE').mean().reset_index(), x='POSTAL.CODE', y='PC.REALGSP.REL')

DC is very high compared to the rest of the states, probably because the statistic is adjusted per capita rather than total. Below is a modificaiton to the data to show the total GDP of each state by multiplying by population.

In [4]:
formated_data['REALGSP'] = formated_data['PC.REALGSP.REL'] * formated_data['POPULATION']
px.bar(formated_data.groupby('POSTAL.CODE')[['REALGSP']].mean().reset_index(), x='POSTAL.CODE', y='REALGSP')

This result makes much more sense and is more in line with the actual wealth of each state. Now, does this correlate with outage times? Likely it would correlate with frequency as it is also directly correlated with population density and population.

In [5]:
px.scatter(formated_data, x='REALGSP', y='OUTAGE.DURATION')

Lots of the durations here are very low, is this because the data is not accurate or because the outages are very short? Compare with something like peak demand loss to see if there is a corelation between the two, which would be expected.

In [6]:
px.scatter(formated_data, x='OUTAGE.DURATION', y='DEMAND.LOSS.MW')

Indeed, an exponental decay relationship is present (For linear model, this may be a useful feature). Lets drop the columns with no peak demand loss and plot the above graph again.

In [7]:
px.scatter(formated_data[formated_data['DEMAND.LOSS.MW'] > 0], x='OUTAGE.DURATION', y='DEMAND.LOSS.MW')

Still looks fairly similar. There are also a number of very high values, which are likely the points where total demand loss was reported instead of peak demand loss. This could be an issue for training a model, as it would unfairly weight these points. Unfortunately, there is no way to tell which points are which.

Exploration of outages by state. The first is an absolute value, while the seccond is normalized against the number of people in the state.

In [8]:
state = formated_data.groupby('POSTAL.CODE').count().reset_index()
px.bar(state, x='POSTAL.CODE', y='OBS', title='State Outages')

In [9]:
capita = formated_data.groupby('POSTAL.CODE')['OBS'].count()
pop = formated_data.groupby('POSTAL.CODE')['POPULATION'].mean()
capita = capita / pop

In [10]:
px.bar(capita, x=capita.index, y=[0], title='Outages per Capita')

In [11]:
capita.median()

3.92684105058557e-06

In [12]:
zcap = pd.Series(index=capita.index, data=stats.zscore(capita))
delz = zcap.loc['DE']
delz

5.723815787163536

Deleware is very interesting here, as it has more than 6 times the mean, and more than twice the number per capita as the seccond highest value. Why might this be?

Exploration of proportions of outages attibuted to weather events year over year

In [13]:
weather_year = formated_data[formated_data['CAUSE.CATEGORY'] == 'severe weather'].groupby('YEAR').count()['OBS'] / formated_data.groupby('YEAR').count()['OBS']
px.scatter(weather_year, x=weather_year.index, y='OBS', title='Severe Weather Outages by Year').show()

In [14]:
weather_year.drop(2001, inplace=True)
stats.pearsonr(weather_year.index, weather_year.values)

PearsonRResult(statistic=-0.7561000778006577, pvalue=0.0007019393734256857)

Dropping the 2001 data point has a massive effect on the r value and the p value of the r correlation. the r value becomes very strongly positive from weakly negative. The p value becomes well below the 0.05 threshold from 0.5. Why?

Absolute number of outages occuring year over year

In [15]:
yearly = formated_data.groupby('YEAR').count().reset_index()
px.scatter(yearly, x='YEAR', y='OBS', title='Yearly Outages')

TODO: Find a bunch of pivot tables with the very short outages, and without them, seeing if you can find a correlation (shorter outages probably have different causes and effects)

In [16]:
filtered = formated_data[formated_data['OUTAGE.DURATION'] < 50]
filtered['OUTAGE.DURATION'] = pd.cut(filtered['OUTAGE.DURATION'].dropna().astype(int), 100)
# pivot = formated_data.pivot_table(index='YEAR', columns='POSTAL.CODE', values='OBS', aggfunc='count')
# pivot.fillna(0)
filtered = filtered.groupby('OUTAGE.DURATION').count().reset_index()
filtered['OUTAGE.DURATION'] = filtered['OUTAGE.DURATION'].astype(str)
px.bar(filtered, x='OUTAGE.DURATION', y='OBS')

/var/folders/v7/nxggzv_j5s936v9rvl85gh2w0000gn/T/ipykernel_94420/3831222645.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Lots of these values seem to be 1 or 0. Look at all of these and see what they have in common

In [17]:
low_durs = formated_data[formated_data['OUTAGE.DURATION'] <= 1]
low_durs.pivot_table(index='OUTAGE.DURATION', columns='ANOMALY.LEVEL', values='DEMAND.LOSS.MW')

ANOMALY.LEVEL,-1.3,-1.1,-1.0,-0.9,-0.8,-0.7,-0.6,-0.5,-0.4,-0.3,-0.1,0.1,0.3,0.6,0.8,1.6,2.3
OUTAGE.DURATION,,,,,,,,,,,,,,,,,
0.0,0.0,1040.0,0.0,1.4,0.0,0.0,0.0,0.000000,0.2,0.000000,0.0,NaN,0.0,NaN,NaN,0.0,NaN
1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,728.333333,157.5,18.333333,NaN,0.0,0.0,12.0,0.0,NaN,0.0


Some of these points dont seem to make any sense - how can an outage lasting 0 minutes cause a peak demand loss of 1040 MW? For most analysis, and prediction, it may make sense to drop these values

In [18]:
filtered = formated_data[formated_data['OUTAGE.DURATION'] > 1]
filtered['OUTAGE.DURATION'] = pd.cut(filtered['OUTAGE.DURATION'].dropna().astype(int), 100)
filtered.pivot_table(index='OUTAGE.DURATION', columns='ANOMALY.LEVEL', values='DEMAND.LOSS.MW')

/var/folders/v7/nxggzv_j5s936v9rvl85gh2w0000gn/T/ipykernel_94420/1580646071.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



ANOMALY.LEVEL,-1.6,-1.5,-1.4,-1.3,-1.2,-1.1,-1.0,-0.9,-0.8,-0.7,...,1.0,1.1,1.2,1.3,1.4,1.6,1.7,2.0,2.2,2.3
OUTAGE.DURATION,,,,,,,,,,,,,,,,,,,,,
"(-106.651, 1088.51]",NaN,258.0,211.666667,534.0,280.333333,378.333333,595.75,303.846154,0.0,157.000000,...,189.333333,177.142857,283.0,350.0,NaN,200.0,75.0,4188.5,0.0,96.0
"(1088.51, 2175.02]",NaN,NaN,464.000000,180.0,NaN,NaN,4000.00,300.000000,NaN,475.333333,...,NaN,0.000000,250.0,2650.0,NaN,0.0,NaN,NaN,0.0,0.0
"(2175.02, 3261.53]",NaN,240.0,331.000000,NaN,NaN,NaN,NaN,240.000000,NaN,600.000000,...,NaN,59.500000,NaN,NaN,1200.0,NaN,NaN,NaN,4.0,NaN
"(3261.53, 4348.04]",NaN,NaN,130.000000,79.0,NaN,NaN,176.50,400.000000,NaN,177.500000,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
"(4348.04, 5434.55]",NaN,NaN,115.500000,NaN,NaN,NaN,91.00,NaN,NaN,200.000000,...,NaN,NaN,270.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(5434.55, 6521.06]",NaN,NaN,NaN,NaN,340.000000,NaN,NaN,540.000000,NaN,637.500000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(6521.06, 7607.57]",375.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,91.000000,...,NaN,180.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(7607.57, 8694.08]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,800.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(8694.08, 9780.59]",NaN,NaN,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,250.0,NaN,NaN,NaN,NaN,NaN


In [19]:
pivoter = formated_data.copy()
pivoter['OUTAGE.DURATION'] = pd.qcut(formated_data['OUTAGE.DURATION'], 10, labels=False)
pivot2 = formated_data.pivot_table(index='OUTAGE.DURATION', columns='NERC.REGION', values='TOTAL.PRICE', aggfunc='mean')
pivot2

NERC.REGION,ECAR,FRCC,"FRCC, SERC",HECO,HI,MRO,NPCC,PR,RFC,SERC,SPP,TRE,WECC
OUTAGE.DURATION,,,,,,,,,,,,,
0.0,NaN,NaN,NaN,NaN,NaN,9.28,14.726154,NaN,10.987187,9.150000,9.56,8.56,7.220769
1.0,NaN,NaN,NaN,NaN,NaN,9.28,15.001538,NaN,10.768421,9.431667,NaN,9.04,9.304444
2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.310000,8.800000,NaN,NaN,8.837500
3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.880000,NaN,6.77,NaN,13.320000
4.0,NaN,NaN,NaN,NaN,NaN,6.20,NaN,NaN,NaN,NaN,NaN,NaN,8.350000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
49320.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.910000
49427.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.340000
60480.0,NaN,NaN,NaN,NaN,NaN,NaN,17.810000,NaN,NaN,NaN,NaN,NaN,NaN


## Step 3: Assessment of Missingness

To get a sense of missingess, make a dataframe with all the datapoints containing a missing value

In [20]:
def missing_points(df: pd.DataFrame):
    hasna = np.repeat(False, df.shape[0])
    for col in df.columns:
        hasna = (hasna | df[col].isna())
    return df.loc[hasna]
missing_all = missing_points(formated_data)

Since the hurricane name column only applies to very few data points, and is therefore nan for most (MD), drop that one to see which datapoints might contain some form of unintentional missingess

In [21]:
no_hur_missing = missing_points(formated_data.drop(columns=['HURRICANE.NAMES']))
no_hur_missing.shape[0]

1039

Find all of the columns that contain any missing data

In [22]:
hasna = np.repeat(False, formated_data.shape[1])
index = 0
for col in formated_data.columns:
    if np.any(formated_data[col].isna()):
        hasna[index] = True
    index += 1
has_missing = formated_data.loc[:, hasna]
has_missing.columns

Index(['MONTH', 'CLIMATE.REGION', 'ANOMALY.LEVEL', 'CLIMATE.CATEGORY',
       'OUTAGE.START.DATE', 'OUTAGE.START.TIME', 'OUTAGE.RESTORATION.DATE',
       'OUTAGE.RESTORATION.TIME', 'CAUSE.CATEGORY.DETAIL', 'HURRICANE.NAMES',
       'OUTAGE.DURATION', 'DEMAND.LOSS.MW', 'CUSTOMERS.AFFECTED', 'RES.PRICE',
       'COM.PRICE', 'IND.PRICE', 'TOTAL.PRICE', 'RES.SALES', 'COM.SALES',
       'IND.SALES', 'TOTAL.SALES', 'RES.PERCEN', 'COM.PERCEN', 'IND.PERCEN',
       'POPDEN_UC', 'POPDEN_RURAL', 'outageStart', 'outageEnd'],
      dtype='object', name=4)

In [23]:
nomissing = formated_data.loc[:, ~hasna]
nomissing.columns

Index(['OBS', 'YEAR', 'U.S._STATE', 'POSTAL.CODE', 'NERC.REGION',
       'CAUSE.CATEGORY', 'RES.CUSTOMERS', 'COM.CUSTOMERS', 'IND.CUSTOMERS',
       'TOTAL.CUSTOMERS', 'RES.CUST.PCT', 'COM.CUST.PCT', 'IND.CUST.PCT',
       'PC.REALGSP.STATE', 'PC.REALGSP.USA', 'PC.REALGSP.REL',
       'PC.REALGSP.CHANGE', 'UTIL.REALGSP', 'TOTAL.REALGSP', 'UTIL.CONTRI',
       'PI.UTIL.OFUSA', 'POPULATION', 'POPPCT_URBAN', 'POPPCT_UC',
       'POPDEN_URBAN', 'AREAPCT_URBAN', 'AREAPCT_UC', 'PCT_LAND',
       'PCT_WATER_TOT', 'PCT_WATER_INLAND', 'REALGSP'],
      dtype='object', name=4)

In [24]:
formated_data['DEMAND.LOSS.MW'].isna()
formated_data['POSTAL.CODE']
postal_loss = formated_data[['POSTAL.CODE', 'DEMAND.LOSS.MW']]
postal_loss.loc[:, 'Missing'] = postal_loss['DEMAND.LOSS.MW'].isna()
postal_loss = postal_loss.groupby('POSTAL.CODE').sum()
postal_loss

/Users/martinhawks/miniconda3/envs/dsc80/lib/python3.8/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



4,DEMAND.LOSS.MW,Missing
POSTAL.CODE,,
AK,35.0,0
AL,583.0,4
AR,1499.0,15
AZ,12457.0,18
CA,105480.0,52
CO,1701.0,4
CT,255.0,11
DC,3840.0,7
DE,95.0,18


Is there perhaps an association between the wealth of the state and the quality of the data? This could be a potential source of bias in the data - if the data is missing in states with lower GDP, the data might have a bias when predicting on certain states.

In [25]:
gsp_postal_missing = postal_loss.merge(formated_data.groupby('POSTAL.CODE')['TOTAL.REALGSP'].mean(), left_index=True, right_index=True)
gsp_postal_missing['Totals'] = formated_data.groupby('POSTAL.CODE').count()['OBS']
gsp_postal_missing['Missing_prop'] = gsp_postal_missing['Missing'] / gsp_postal_missing['Totals']
px.scatter(gsp_postal_missing, x='TOTAL.REALGSP', y='Missing_prop', title='Missing Demand Loss by Real GSP')

Perhaps a weak correlation? Run a test to see if the correlation is significant.

In [26]:
stats.pearsonr(gsp_postal_missing['TOTAL.REALGSP'], gsp_postal_missing['Missing_prop'])

PearsonRResult(statistic=0.01487506360937882, pvalue=0.9183377304861762)

Clearly, there is no correlation between missingness of the peak demand loss and the total GSP of each state.

test against NERC region as well

In [28]:
missing_nerc = formated_data.copy()
missing_nerc.loc[:, 'NERC'] = missing_nerc['DEMAND.LOSS.MW'].isna()
missing_nerc = missing_nerc.groupby('NERC.REGION').sum()
missing_nerc['Missing_prop'] = missing_nerc['NERC'] / formated_data.groupby('NERC.REGION').count()['OBS']
missing_nerc = missing_nerc.merge(formated_data.groupby('NERC.REGION')['OBS'].count(), left_index=True, right_index=True)
missing_nerc[['Missing_prop', 'OBS_x']]

,Missing_prop,OBS_x
NERC.REGION,,
ASCC,0.000000,1534
ECAR,0.088235,14024
FRCC,0.090909,45540
"FRCC, SERC",1.000000,1047
HECO,0.000000,4557
HI,0.000000,1516
MRO,0.565217,19137
NPCC,0.513333,180347
PR,0.000000,1517


This is much more interesting - some regions have no missing data, while others have a lot. This seems a lot more like a correlation. Also, there seems to be a data point which exists in two regions - why might this be?

In [29]:
def tvd(s1, s2):
    return np.abs(s1 - s2).sum() / 2

test = formated_data[['DEMAND.LOSS.MW', 'NERC.REGION', 'OBS']]
N = 10_000
tvds = np.repeat(0.0, N)
for i in range(N):
    test['Shuffled'] = np.random.permutation(test['DEMAND.LOSS.MW'].isna())
    grouped = test.groupby('NERC.REGION').sum()
    grouped.loc[:, 'Missing_prop'] = grouped['Shuffled'] / test.groupby('NERC.REGION').count()['OBS']
    grouped.loc[:, 'non_missing_prop'] = 1 - grouped['Missing_prop']
    tvds[i] = tvd(grouped['Missing_prop'], grouped['non_missing_prop'])

obs = tvd(missing_nerc['Missing_prop'], 1 - missing_nerc['Missing_prop'])
p = np.mean(tvds >= obs)
p

/var/folders/v7/nxggzv_j5s936v9rvl85gh2w0000gn/T/ipykernel_94420/4157915321.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



0.0

Given this P value, it would seem that the missingess of the Peak loss column is very much MAR dependent on the region that the outage occured in. Keep note, as this could be a source of bias in the data.

## Step 4: Hypothesis Testing

Analysis of outages caused by weather events vs. outages caused by non-weather events

In [30]:
weather = (formated_data[formated_data['CAUSE.CATEGORY'] == 'severe weather'].groupby('YEAR').count().sort_values('OBS', ascending=False) / formated_data.groupby('YEAR').count())
px.scatter(weather, x=weather.index, y='OBS', title='Severe Weather Outages by Year')

In [31]:
weather

4,OBS,MONTH,U.S._STATE,POSTAL.CODE,NERC.REGION,CLIMATE.REGION,ANOMALY.LEVEL,CLIMATE.CATEGORY,OUTAGE.START.DATE,OUTAGE.START.TIME,...,POPDEN_UC,POPDEN_RURAL,AREAPCT_URBAN,AREAPCT_UC,PCT_LAND,PCT_WATER_TOT,PCT_WATER_INLAND,outageStart,outageEnd,REALGSP
YEAR,,,,,,,,,,,,,,,,,,,,,
2000,0.500000,0.526316,0.500000,0.500000,0.500000,0.520000,0.526316,0.526316,0.526316,0.526316,...,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.692308,0.692308,0.500000
2001,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,...,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.071429,0.071429,0.066667
2002,0.764706,0.750000,0.764706,0.764706,0.764706,0.764706,0.750000,0.750000,0.750000,0.750000,...,0.764706,0.764706,0.764706,0.764706,0.764706,0.764706,0.764706,0.785714,0.785714,0.764706
2003,0.652174,0.652174,0.652174,0.652174,0.652174,0.652174,0.652174,0.652174,0.652174,0.652174,...,0.644444,0.644444,0.652174,0.652174,0.652174,0.652174,0.652174,0.652174,0.652174,0.652174
2004,0.788732,0.788732,0.788732,0.788732,0.788732,0.788732,0.788732,0.788732,0.788732,0.788732,...,0.788732,0.788732,0.788732,0.788732,0.788732,0.788732,0.788732,0.788732,0.788732,0.788732
2005,0.854545,0.854545,0.854545,0.854545,0.854545,0.854545,0.854545,0.854545,0.854545,0.854545,...,0.854545,0.854545,0.854545,0.854545,0.854545,0.854545,0.854545,0.870370,0.870370,0.854545
2006,0.805970,0.818182,0.805970,0.805970,0.805970,0.812500,0.818182,0.818182,0.818182,0.818182,...,0.805970,0.805970,0.805970,0.805970,0.805970,0.805970,0.805970,0.818182,0.818182,0.805970
2007,0.714286,0.714286,0.714286,0.714286,0.714286,0.714286,0.714286,0.714286,0.714286,0.714286,...,0.714286,0.714286,0.714286,0.714286,0.714286,0.714286,0.714286,0.740741,0.740741,0.714286
2008,0.684685,0.684685,0.684685,0.684685,0.684685,0.681818,0.684685,0.684685,0.684685,0.684685,...,0.684685,0.684685,0.684685,0.684685,0.684685,0.684685,0.684685,0.690909,0.690909,0.684685


In [32]:
no_drop = stats.pearsonr(weather.index, weather['OBS'])
weather.drop(2001, inplace=True)
drop = stats.pearsonr(weather.index, weather['OBS'])
no_drop, drop

(PearsonRResult(statistic=-0.40759750516416526, pvalue=0.10437683084481637),
 PearsonRResult(statistic=-0.7561000778006577, pvalue=0.0007019393734256857))

Interesting how the one datapoint has such a massive effect on the correlation. Why might this be?

Correlation between peak demand loss and total cost of electricity in the area

In [33]:
# Start by getting the relevant columns, and dropping nan vals
temp_data = formated_data[['TOTAL.PRICE', 'DEMAND.LOSS.MW', 'OUTAGE.DURATION']].dropna()
temp_data['OUTAGE.DURATION'] = temp_data['OUTAGE.DURATION'].astype(float)
temp_data.loc[:, 'DEMAND.LOSS.MW'] = temp_data['DEMAND.LOSS.MW'].astype(float)
# temporarily replace 0 with nan to avoid log(0) error
temp_data.loc[temp_data['DEMAND.LOSS.MW'] == 0, 'DEMAND.LOSS.MW'] = np.nan
temp_data.loc[:, 'DEMAND.LOSS.MW'] = np.log(temp_data['DEMAND.LOSS.MW']).fillna(0)
px.scatter(temp_data, x='TOTAL.PRICE', y='DEMAND.LOSS.MW', title='Cost vs Demand Loss', color='OUTAGE.DURATION')

In [34]:
stats.pearsonr(temp_data['TOTAL.PRICE'], temp_data['DEMAND.LOSS.MW'])

PearsonRResult(statistic=-0.07534448817021411, pvalue=0.033552353895290224)

## Step 5: Framing a Prediction Problem

We will be training a model to predict the outage duration of a power outage, idealy to make a prediction after the power is lost. Thus, features like customers affected will be accesible to the model, but features like peak demand loss will not be, since that feature is dependent on the length of the outage

In [35]:
def un_capita(df: pd.DataFrame):
    out = (df['PC.REALGSP.REL'] * df['POPULATION'])
    out.name = 'un_capita'
    return out

def prev(df: pd.DataFrame):
    df = df.sort_values(by='outageStart')
    df['prev_out'] = np.arange(0, df.shape[0])
    return df.drop(columns=['POSTAL.CODE'])

In [36]:
formated_data = formated_data.groupby('POSTAL.CODE').apply(prev).reset_index().drop(columns='level_1')

In [37]:
formated_data['nopcGSPRel'] = formated_data.groupby('POSTAL.CODE').apply(un_capita).reset_index(drop=True)
px.scatter(formated_data, x='prev_out', y='OUTAGE.DURATION', title='Previous outages vs Outage Duration')

In [38]:
stats.pearsonr(formated_data.dropna()['prev_out'], formated_data.dropna()['OUTAGE.DURATION'])

PearsonRResult(statistic=0.35598595747726314, pvalue=0.02235190602694181)

We will drop the 0 and 1 values in the outage duration column, since any outage that is that short will not be long enough for anyone to obtain the information about the outage and supply it to the model.

In [39]:
training_data = formated_data[formated_data['OUTAGE.DURATION'] > 1]
training_data = training_data.dropna(subset=['OUTAGE.DURATION'])
training_data['OUTAGE.DURATION'] = training_data['OUTAGE.DURATION'].astype(float)
training_data = training_data.drop(columns=['OBS', 'DEMAND.LOSS.MW'])
px.histogram(training_data, x='OUTAGE.DURATION')

The trouble with this prediction problem is the very high outliers. Since there are so few, we will drop them for finding features to train the model on.

In [40]:
training_data['DurationCut'] = pd.qcut(training_data['OUTAGE.DURATION'], np.linspace(0, 1, 101), labels=np.linspace(0.01, 1, 100)).astype(float)
training_data = training_data[training_data['DurationCut'] < 0.95]
px.histogram(training_data, x='OUTAGE.DURATION')

A more distributed interval, which will be easier to find correlations with

In [41]:
px.scatter(training_data, x='TOTAL.PRICE', y='CUSTOMERS.AFFECTED', color='OUTAGE.DURATION')

We can see again an interesting number of outages which seemed to affect no customerWe can see again an interesting number of outages which seemed to affect no customers. Again, we will drop the 0 values from the training dataset, since an outage that has no affect doesn't seem to be much of an actual power outage

In [42]:
training_data = training_data[training_data['CUSTOMERS.AFFECTED'] > 0]
px.scatter(training_data, x='TOTAL.PRICE', y='CUSTOMERS.AFFECTED', color='OUTAGE.DURATION', title='Price and Affected clusters')

In [43]:
px.histogram(training_data, x='CUSTOMERS.AFFECTED')

In [44]:
px.bar(training_data.groupby('CAUSE.CATEGORY').mean().reset_index(), x='CAUSE.CATEGORY', y='OUTAGE.DURATION', title='Outages by Cause')

Lots of outliers seem to exist in this dataset, and the way that the price vs affected clusters seem to group looks as though a DesisionTree/RandomForest regressor would be good for making predicitons here

It also seems like there are certain causes that cause longer outage times, so oneHotEncoding this information will likely also be a good feature to include in the model

In [45]:
peek = formated_data.groupby('CAUSE.CATEGORY').count()
look = peek[['TOTAL.PRICE', 'prev_out', 'CUSTOMERS.AFFECTED', 'OUTAGE.DURATION']].loc['fuel supply emergency']
look

4
TOTAL.PRICE           50
prev_out              51
CUSTOMERS.AFFECTED     7
OUTAGE.DURATION       38
Name: fuel supply emergency, dtype: int64

Note that the values we want to train on contain mostly nan values for the fuel supply emergency column - find a way to impute this so that there is enough data of this type to train the model on

## Step 6: Baseline Model

For out Baseline Model, we will start by using a RandomForestRegressor, including the Customers Affected and a OneHotEncoding of the Cause catagory, trained via a gridsearch. For this baseline, we will not try to find perfect parameters, simply look over a couple of spaced ones to find an ideal outcome. Scoring will be assesed using a 10 fold cross validation on the entire avaliable dataset

In [63]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import (train_test_split, cross_val_score)
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold

# Get training features
training_attrs = [
    'CAUSE.CATEGORY', 'TOTAL.PRICE', 'prev_out', 'CUSTOMERS.AFFECTED', 'OUTAGE.DURATION'
    ]
tmp_formater = formated_data[training_attrs].dropna()
tmp_formater = tmp_formater[tmp_formater['CUSTOMERS.AFFECTED'] > 0]
X = tmp_formater[['CAUSE.CATEGORY', 'TOTAL.PRICE', 'prev_out', 'CUSTOMERS.AFFECTED']]
y = tmp_formater['OUTAGE.DURATION']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [47]:
# Make the column transformer
trans = ColumnTransformer([
    ('onehot', OneHotEncoder(handle_unknown='ignore'), ['CAUSE.CATEGORY']),  # handle unknown is ignore, as there is 
    # one category that has only 1 instance in the training data
], remainder='passthrough')
# Make a Pipeline

pipe = Pipeline([
    ('transform', trans),
    ('model', RandomForestRegressor())
])

param_grid = {
    'model__n_estimators': np.arange(5, 60, 5),
    'model__max_depth': np.arange(2, 15, 5)
}
searcher = GridSearchCV(
    pipe, param_grid, cv=5, n_jobs=-1, scoring='r2',
)
searcher.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('transform',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('onehot',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['CAUSE.CATEGORY'])])),
                                       ('model', RandomForestRegressor())]),
             n_jobs=-1,
             param_grid={'model__max_depth': array([ 2,  7, 12]),
                         'model__n_estimators': array([ 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55])},
             scoring='r2')

In [48]:
# Score the model
searcher.score(X_test, y_test)

0.15989902137180378

Not Ideal... See what can be imporoved in the next steps

## Step 7: Final Model

One issue with the old model is that many of the missing values seems to be highly correlated with being a fuel supply emergency, which also seems to indicate much higher outage times. Becasue of the missingness, the model isn't able to make use of this as well, so we will impute some missing values so we can still use these data points

In [64]:
peek_nans = formated_data[training_attrs]
peek_nans.isna().sum()

4
CAUSE.CATEGORY          0
TOTAL.PRICE            22
prev_out                0
CUSTOMERS.AFFECTED    443
OUTAGE.DURATION        58
dtype: int64

Imputing total price shouldn't be too dificult - it is likely correlated with the geographic region **PROVE**, so we can group by that and use probabalistic imputation to fil the missing values. 

In [75]:
def prob_imp(ser: pd.Series):
    nans = ser[ser.isna()]
    if nans.shape[0] == 0 or nans.shape[0] == ser.shape[0]:
        return ser
    newvals = np.random.choice(ser.dropna(), nans.shape[0])
    ser.loc[nans.index] = newvals
    return ser

imp_nans = formated_data.copy()

imp_nans['TOTAL.PRICE'] = imp_nans.groupby('NERC.REGION')['TOTAL.PRICE'].transform(prob_imp)

In [76]:
imp_nans[training_attrs].isna().sum()

4
CAUSE.CATEGORY          0
TOTAL.PRICE             1
prev_out                0
CUSTOMERS.AFFECTED    443
OUTAGE.DURATION        58
dtype: int64

Why there are so many customers affected values missing is another interesting question, and it has less of an obivous answer.

In [77]:
def prop_nan(ser: pd.Series):
    return ser.isna().sum() / ser.shape[0]

imp_nans.pivot_table(index='CAUSE.CATEGORY', values='OUTAGE.DURATION', aggfunc=prop_nan)

4,OUTAGE.DURATION
CAUSE.CATEGORY,
equipment failure,0.083333
fuel supply emergency,0.254902
intentional attack,0.035885
islanding,0.043478
public appeal,0.000000
severe weather,0.024902
system operability disruption,0.031496


Of all of the Categorical Columns, Cause category seems to have the highest correlation with the missingess of Outage.Duration, so we will use that as the grouping to impute with

**USE POP_DENSITY TO IMPUTE**

In [78]:
imp_nans['CUSTOMERS.AFFECTED'] = imp_nans.groupby('CAUSE.CATEGORY')['OUTAGE.DURATION'].transform(prob_imp)
imp_nans[training_attrs].isna().sum()

4
CAUSE.CATEGORY         0
TOTAL.PRICE            1
prev_out               0
CUSTOMERS.AFFECTED     0
OUTAGE.DURATION       58
dtype: int64

In [ ]:
new_X = imp_nans[training_attrs].dropna()
new_y = new_X['OUTAGE.DURATION']
new_X = new_X.drop(columns='OUTAGE.DURATION')

X_train, X_test, y_train, y_test = train_test_split(new_X, new_y, test_size=0.2, random_state=0)

In [79]:
# We will refit our old model to the new data
searcher.fit(X_train, y_train)
searcher.score(X_test, y_test)

0.18848139087514293

A slight improvement, but not by a whole lot. Lets see if we can use another model to improve

In [80]:
from sklearn.linear_model import LinearRegression

lin_pipe = Pipeline([
    ('transform', trans),
    ('model', LinearRegression())
])

param_grid = {
    'model__fit_intercept': [True, False],
    # 'model__normalize': [True, False]
}
lin_searcher = GridSearchCV(
    lin_pipe, param_grid, cv=5, n_jobs=-1, scoring='r2',
)
lin_searcher.fit(X_train, y_train)
lin_searcher.score(X_test, y_test)

0.17793811790715275

Doesn't seem to be as good as using the linear RandomForest. Lets try some other models

In [81]:
from sklearn.linear_model import Ridge

ridge_pipe = Pipeline([
    ('transform', trans),
    ('model', Ridge())
])
param_grid = {
    'model__alpha': np.linspace(0.1, 10, 100)
}
ridge_searcher = GridSearchCV(
    ridge_pipe, param_grid, cv=5, n_jobs=-1, scoring='r2',
)
ridge_searcher.fit(X_train, y_train)
ridge_searcher.score(X_test, y_test)

0.17720753614265616

Seems like this is either not a good set of features to train a linear model on. We'll try adding some new ones

In [87]:
training_attrs

['CAUSE.CATEGORY',
 'TOTAL.PRICE',
 'prev_out',
 'CUSTOMERS.AFFECTED',
 'OUTAGE.DURATION']

In [121]:
from sklearn.preprocessing import StandardScaler


new_params = training_attrs + ['TOTAL.SALES', 'TOTAL.CUSTOMERS']
new_trans = ColumnTransformer([
    ('onehot', OneHotEncoder(handle_unknown='ignore'), ['CAUSE.CATEGORY']),
    # handle unknown is ignore, as there is 
    # one category that has only 1 instance in the training data
    ('scale', StandardScaler(), ['TOTAL.SALES', 'TOTAL.CUSTOMERS', 'TOTAL.PRICE'])],
    remainder='passthrough')

new_pipe = Pipeline([
    ('transform', new_trans),
    ('model', RandomForestRegressor())
])
new_X = formated_data[new_params].dropna()
new_y = new_X['OUTAGE.DURATION']
new_X = new_X.drop(columns='OUTAGE.DURATION')

X_train, X_test, y_train, y_test = train_test_split(new_X, new_y, test_size=0.2, random_state=0)

param_grid = {
    'model__n_estimators': np.arange(5, 60, 5),
    'model__max_depth': np.arange(2, 15, 5)
}
new_searcher = GridSearchCV(
    new_pipe, param_grid, cv=5, n_jobs=-1, scoring='r2',
)
new_searcher.fit(X_train, y_train)
new_searcher.score(X_test, y_test)


0.300797625196461

In [122]:
new_searcher.best_params_

{'model__max_depth': 7, 'model__n_estimators': 45}

Definite improvement by adding more parameters. Some of them are sums of multiple, so we will see if using their component parts also improves the accuracy

In [123]:
training_attrs

['CAUSE.CATEGORY',
 'TOTAL.PRICE',
 'prev_out',
 'CUSTOMERS.AFFECTED',
 'OUTAGE.DURATION']

In [124]:
split_feats = [
    'CAUSE.CATEGORY', 'prev_out', 'CUSTOMERS.AFFECTED',
    'OUTAGE.DURATION', 'RES.PRICE', 'COM.PRICE', 'IND.PRICE', 'RES.SALES',
    'COM.SALES', 'IND.SALES'
    ]
split_trans = ColumnTransformer(
    [
    ('onehot', OneHotEncoder(handle_unknown='ignore'), ['CAUSE.CATEGORY']),
    # handle unknown is ignore, as there is 
    # one category that has only 1 instance in the training data
    ('scale', StandardScaler(), [
        'RES.PRICE', 'COM.PRICE', 'IND.PRICE', 'RES.SALES', 'COM.SALES', 'IND.SALES'
        ])],
    remainder='passthrough'
)
split_pipe = Pipeline([
    ('transform', split_trans),
    ('model', RandomForestRegressor())
])
split_X = formated_data[split_feats].dropna()
split_y = split_X['OUTAGE.DURATION']
split_X = split_X.drop(columns='OUTAGE.DURATION')

X_train, X_test, y_train, y_test = train_test_split(split_X, split_y, test_size=0.2, random_state=0)

split_searcher = GridSearchCV(
    split_pipe, param_grid, cv=5, n_jobs=-1, scoring='r2',
)
split_searcher.fit(X_train, y_train)
split_searcher.score(X_test, y_test)

0.31479282770696637

In [92]:
split_searcher.best_params_

{'model__max_depth': 7, 'model__n_estimators': 25}

Areas with higher population density probably are higher in the priority of places to get serviced, so we will also include that in our model. This may not do a whole lot, since we already have a customers affected column, but areas with higher populaiton density probably have better infrastructure than places without

In [97]:
dens_feats = split_feats + ['AREAPCT_URBAN']
dens_trans = ColumnTransformer(
    [
    ('onehot', OneHotEncoder(handle_unknown='ignore'), ['CAUSE.CATEGORY']),
    # handle unknown is ignore, as there is 
    # one category that has only 1 instance in the training data
    ('scale', StandardScaler(), [
        'RES.PRICE', 'COM.PRICE', 'IND.PRICE', 'RES.SALES', 'COM.SALES', 'IND.SALES'
        ])],
    remainder='passthrough'
)
dens_pipe = Pipeline([
    ('transform', split_trans),
    ('model', RandomForestRegressor())
])
dens_X = formated_data[dens_feats].dropna()
dens_y = dens_X['OUTAGE.DURATION']
dens_X = dens_X.drop(columns='OUTAGE.DURATION')

X_train, X_test, y_train, y_test = train_test_split(split_X, split_y, test_size=0.2, random_state=0)

dens_searcher = GridSearchCV(
    dens_pipe, param_grid, cv=5, n_jobs=-1, scoring='r2',
)
dens_searcher.fit(X_train, y_train)
dens_searcher.score(X_test, y_test)


0.3028212593405678

Doesn't seem to improve the model at all, so probably population encodes this information just as well. Adding weather information seems to also produce some overfitting of the training dataset, so we tried multiplying the ANOMOLY.LEVEL with PCT_WATER_TOT column, with the idea that ares containing more water would be more affected by a high weather event, which would in turn make fixing outages harder.

In [113]:
weather_feats = split_feats + ['ANOMALY.LEVEL', 'PCT_WATER_TOT']

weather_trans = ColumnTransformer(
    [
    ('onehot', OneHotEncoder(handle_unknown='ignore'), ['CAUSE.CATEGORY']),
    # handle unknown is ignore, as there is 
    # one category that has only 1 instance in the training data
    ('scale', StandardScaler(), [
        'RES.PRICE', 'COM.PRICE', 'IND.PRICE', 'RES.SALES', 'COM.SALES', 'IND.SALES'
        ])],
    remainder='passthrough'
)
weather_pipe = Pipeline([
    ('transform', weather_trans),
    ('model', RandomForestRegressor())
])
weather_X = formated_data[weather_feats].dropna()
weather_y = weather_X['OUTAGE.DURATION']
weather_X['SYNTH_WEATHER'] = weather_X['ANOMALY.LEVEL'] * weather_X['PCT_WATER_TOT']
weather_X = weather_X.drop(columns=['OUTAGE.DURATION', 'ANOMALY.LEVEL', 'PCT_WATER_TOT'])

X_train, X_test, y_train, y_test = train_test_split(weather_X, weather_y, test_size=0.2, random_state=0)

weather_searcher = GridSearchCV(
    weather_pipe, param_grid, cv=5, n_jobs=-1, scoring='r2',
)
weather_searcher.fit(X_train, y_train)
weather_searcher.score(X_test, y_test)

0.3087401885797256

At this point, we seem to be reaching a point where adding more features only produces overfitting of the model, rather than improving it's testing accuracy. We will now move to try and fit different models to the data

In [115]:
from sklearn.ensemble import GradientBoostingRegressor

gbr_pipe = Pipeline([
    ('transform', weather_trans),
    ('model', GradientBoostingRegressor())
])

param_grid = {
    'model__n_estimators': np.arange(5, 60, 5),
    'model__max_depth': np.arange(2, 15, 5),
    'model__min_samples_split': np.arange(2, 15, 5)
    # 'model__'
}

gbr_searcher = GridSearchCV(
    gbr_pipe, param_grid, cv=5, n_jobs=-1, scoring='r2',
)
gbr_searcher.fit(X_train, y_train)
gbr_searcher.score(X_test, y_test)

0.2345825652368334

In [127]:
from sklearn.linear_model import LinearRegression

lin_pipe = Pipeline([
    ('transform', weather),
    ('model', LinearRegression())
])

param_grid = {
    'model__fit_intercept': [True, False],
    # 'model__normalize': [True, False]
}
lin_searcher = GridSearchCV(
    lin_pipe, param_grid, cv=5, n_jobs=-1, scoring='r2',
)
lin_searcher.fit(X_train, y_train)
lin_searcher.score(X_test, y_test)

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

## Step 8: Fairness Analysis

In [90]:
# TODO